# Convert Bin Files to PNGs

Frames are save in 200-frame .bin files to increase speed during collection. Before analyzing, its faster to save a whole set of .bin files to images.  

Set this to run and leave it. It takes a LONG time.

## Fix Permissions
When collecting, jupyter has to run as SUDO for keyboard communication. This makes some files and folders with sudo permissions, even for read. 

In Terminal:
'sudo chmod -R 777 /data_repo/bravo'

## Dependencies

In [1]:
import os, stat
import numpy as np
import matplotlib.pyplot as plt
import imp

#import utils.run_analysis as ana

## Parameters

*** NOTE: REALLY NEED TO CHECK ON THE VALUES FOR HORIZONTAL AND VERTICAL FOV ***

In [2]:
base_dir = '/hmet_data/calibration/'
data_dir = os.path.join(base_dir, 'raw')
png_dir = os.path.join(base_dir, 'pngs')

fps = 200
img_dims=(1544,2064)

save_batchsize = 400

## Specify Directories

In [3]:
import pandas as pd

trial_list_file = './trial_lists/trial_list_calibration_2.csv'

trials = pd.read_csv(trial_list_file)
trials

,folder,trial,task,iter,subject,calibration_trial,trace_key,wb_trial,start_sec,end_sec,notes
0,2020_11_24,0,calibration,2,bu,NaN,NaN,NaN,0,NaN,NaN


In [4]:
import cv2
def convert_bin_pngs(filename, first_fnum, save_batchsize, save_folder,  dims=(1544,2064)):
    '''
    Take a file saved in .bin format from a ximea camera, and convert it to png images.
    Parameters:
        filename (str): file to be converted
        save_folder (str): folder to save png files
        im_shape (2pule ints): shape of image
        img_format (str): Image format files are saved
    Returns:
        None
    '''
    nbytes = np.prod(dims)
    
    with open(filename, 'rb') as fn:
        bs = fn.read(1)
        for i in range(first_fnum, first_fnum+save_batchsize):
            save_filepath = os.path.join(save_folder, f'frame_{i}.png')
            binary_img = []
            for b in range(nbytes):
                binary_img.append(int.from_bytes(fn.read(1),'big'))
            binary_img = np.array(binary_img)
            cimage = cv2.flip(cv2.cvtColor(np.uint8(binary_img.reshape(dims)),cv2.COLOR_BayerGR2BGR),-1)
            cv2.imwrite(save_filepath, cimage)


In [5]:
from pathlib import Path
def convert_trial_directory(data_dir, png_dir, trial_list_file, idx, save_batchsize):
    ''' Convert a single trial from .bin to .png images This runs for a LONG time'''
    
    trials = pd.read_csv(trial_list_file)
    line = trials.iloc[idx]
    
    bin_folder = os.path.join(data_dir,
                          line['folder'],
                          str(line['trial']).zfill(3),
                          'ximea', 'ximea')
    png_folder = os.path.join(png_dir,
                          line['task'],
                          line['subject'],
                          str(line['iter'])
                         )
    Path(png_folder).mkdir(parents=True, exist_ok=True)
    
    #cam_folder = os.path.join(analysis_folder,camera,'pngs')
        
    print(f'Converting bin to png for folders {bin_folder, png_folder}')
    print(f'Each * is {save_batchsize} frames...')
    frame_start = 0
    bin_file = os.path.join(bin_folder,f'frames_{frame_start}_{frame_start+save_batchsize-1}.bin')
    print(bin_file)
    while(os.path.isfile(bin_file)):
        print('*')
        convert_bin_pngs(bin_file, frame_start, save_batchsize, png_folder, dims=(1544,2064))
        frame_start += save_batchsize
        bin_file = os.path.join(bin_folder,f'frames_{frame_start}_{frame_start+save_batchsize-1}.bin')
    print('Done!')

In [6]:
convert_trial_directory(data_dir, png_dir, trial_list_file, 0, save_batchsize)

Converting bin to png for folders ('/hmet_data/calibration/raw/2020_11_24/000/ximea/ximea', '/hmet_data/calibration/pngs/calibration/bu/2')
Each * is 400 frames...
/hmet_data/calibration/raw/2020_11_24/000/ximea/ximea/frames_0_399.bin
*
*
*
*
Done!
